<h3>Demonstrate the use of MotifsFinder</h3>

In [13]:
import numpy as np
import pandas as pd
from Bio import Align
from Motifstools.FindMotifs import FindMotifs

Input examples. The files with the sequences must be in the Input directory in the format gene_name.fasta.
The genes in sample_list_at/vv are the one that sould be homologues from literature.
The task is one of the Phd thesis.

In [5]:
sample_input_list = ["meme_ara","meme_ara_1","meme_ara_2","meme_ara_3","meme_ara_4","meme_ara_5","meme_ara_6","meme_ara_7","meme_ara","meme_ara_1","meme_ara_2","meme_ara_3","meme_ara_4","meme_ara_5","meme_ara_6","meme_ara_7"]
sample_list_at = ["AT2G46680","AT4G26080","AT3G19290","AT1G08810","AT2G47160"]
sample_list_vv = ["VIT_15s0048g02870","VIT_11s0016g03180","VIT_18s0001g10450","VIT_08s0056g00800","VIT_17s0000g08530"]
sample_task_at = "AT4G28110"
sample_task_vv = ["VIT_12s0134g00570","VIT_19s0014g03820","VIT_00s0203g00070"]

Used for alignment

In [7]:
at_cdna = pd.read_csv("Input/arabidopsis_cDNA.csv",sep="\t")
vv_cdna = pd.read_csv("Input/vitis_cDNA.csv",sep="\t")

Used for some exmaples below

In [8]:
def align(seq1,seq2):
    aligner = Align.PairwiseAligner()
    aligner.mode = "local"
    aligner.open_gap_score = -4
    aligner.extend_gap_score = -2
    aligner.match = 2
    aligner.mismatch = -3
    score = aligner.score(seq1, seq2)
    return score 

In [9]:
def jaccard_similarity(x,y):
    inter = len(set.intersection(set(x), set(y)))
    return inter / float(len(x) + len(y) - inter)

Compute the motifs for two example lists (should be the expansion lists)

In [10]:
at = FindMotifs(sample_list_at)
vv = FindMotifs(sample_list_vv)

Compute some values as testing.<br>
sim_j: matrix 5x5 with only jaccard distances<br>
sim_a: matrix 5x5 with only alignment<br>
sim: matrix 5x5 with jaccard*alignmet<br>

In [11]:
sim_j = np.zeros((5,5))
for i in range(len(at)):
    for j in range(len(vv)):
        sim_j[i,j] = jaccard_similarity(at[i],vv[j])

In [14]:
sim_a = np.zeros((5,5))
for i in range(5):
    for j in range(5):
        a = at_cdna.loc[(at_cdna["a_thaliana_gene"] == sample_list_at[i]),"a_sequence"].iloc[0]
        v = vv_cdna.loc[(vv_cdna["vitis_gene"] == sample_list_vv[j]),"v_sequence"].iloc[0]
        sim_a[i,j] = align(a,v)

In [15]:
sim = np.zeros((5,5))
for i in range(len(at)):
    for j in range(len(vv)):
        a = at_cdna.loc[(at_cdna["a_thaliana_gene"] == sample_list_at[i]),"a_sequence"].iloc[0]
        v = vv_cdna.loc[(vv_cdna["vitis_gene"] == sample_list_vv[j]),"v_sequence"].iloc[0]
        sim[i,j] = jaccard_similarity(at[i],vv[j]) * align(a,v)

In [17]:
print("Jaccard")
print(sim_j)
print("Align")
print(sim_a)
print("Both")
print(sim)

Jaccard
[[0.15062112 0.11735537 0.11050725 0.11150235 0.08876812]
 [0.10895361 0.10689655 0.09756098 0.09982175 0.09225092]
 [0.13333333 0.1037594  0.15292096 0.12541993 0.1520979 ]
 [0.13880597 0.1442623  0.11599297 0.10869565 0.12658228]
 [0.11469933 0.09218203 0.09125    0.13014981 0.14647137]]
Align
[[ 191.   31.   28.   34.   23.]
 [  26.  244.   28.   29.   28.]
 [  25.   35.  255.   28.   26.]
 [  31.   28.   27.  390.   28.]
 [  30.   27.   29.   29. 1163.]]
Both
[[ 28.76863354   3.63801653   3.0942029    3.79107981   2.04166667]
 [  2.83279396  26.08275862   2.73170732   2.89483066   2.58302583]
 [  3.33333333   3.63157895  38.99484536   3.51175812   3.95454545]
 [  4.30298507   4.03934426   3.13181019  42.39130435   3.5443038 ]
 [  3.44097996   2.48891482   2.64625      3.77434457 170.34620506]]


Compute the ratio between the max and the other of a line of the matrix to see if things get better from sim_j/a to sim

In [19]:
print("Jaccard")
print(sim_j[0,:] / max(sim_j[0,:]))
print("Align")
print(sim_a[0,:] / max(sim_a[0,:]))
print("Both")
print(sim[0,:] / max(sim[0,:]))

Jaccard
[1.         0.77914288 0.73367698 0.74028363 0.58934708]
Align
[1.         0.16230366 0.14659686 0.17801047 0.12041885]
Both
[1.         0.12645775 0.10755474 0.13177824 0.0709685 ]


Test for task

In [22]:
task_at = FindMotifs([sample_task_at])
task_vv = FindMotifs(sample_task_vv)

In [24]:
print(jaccard_similarity(task_at[0],task_vv[0]))
print(jaccard_similarity(task_at[0],task_vv[1]))
print(jaccard_similarity(task_at[0],task_vv[2]))

0.14007308160779536
0.14015748031496064
0.15325248070562295
